In [8]:
# Sezione 1: Import librerie
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [9]:
# Sezione 2: Caricamento dati (MovieLens 100k come esempio)
url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_csv(url, sep='\t', names=columns)

# Rimuoviamo timestamp
df.drop('timestamp', axis=1, inplace=True)

In [10]:
# Sezione 3: Creazione matrice utente-film
ratings_matrix = df.pivot_table(index='user_id', columns='movie_id', values='rating')
ratings_matrix.fillna(0, inplace=True)

# Sezione 4: Split del dataset
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [11]:
# Sezione 5: Calcolo similarità tra utenti
user_similarity = cosine_similarity(ratings_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=ratings_matrix.index, columns=ratings_matrix.index)

# Funzione per raccomandare film a un utente basandosi sugli utenti simili
def recommend_movies(user_id, num_recommendations=5):
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]  # escludi se stesso
    weighted_ratings = np.zeros(ratings_matrix.shape[1])
    
    for other_user, similarity in similar_users.items():
        weighted_ratings += similarity * ratings_matrix.loc[other_user].values
    
    user_rated = ratings_matrix.loc[user_id].values > 0
    weighted_ratings[user_rated] = 0  # Escludi quelli già visti
    recommended_indices = np.argsort(weighted_ratings)[::-1][:num_recommendations]
    
    return ratings_matrix.columns[recommended_indices]

/opt/anaconda3/envs/FDSML/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/FDSML/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/opt/anaconda3/envs/FDSML/lib/python3.10/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
# Sezione 6: Costruzione predizioni binarie per valutazione
threshold = 3  # consideriamo raccomandazioni positive se rating >= 3

def predict_rating(user_id, movie_id):
    if movie_id not in ratings_matrix.columns:
        return 0
    similar_users = user_similarity_df[user_id].sort_values(ascending=False)[1:]
    
    num = 0
    den = 0
    for other_user, similarity in similar_users.items():
        rating = ratings_matrix.loc[other_user, movie_id]
        if rating > 0:
            num += similarity * rating
            den += similarity
    if den == 0:
        return 0
    return num / den

In [13]:
# Sezione 7: Valutazione
y_true = []
y_pred = []

test_sample = test_data.sample(1000, random_state=1)

for _, row in test_sample.iterrows():
    actual = 1 if row['rating'] >= threshold else 0
    predicted_rating = predict_rating(row['user_id'], row['movie_id'])
    predicted = 1 if predicted_rating >= threshold else 0
    
    y_true.append(actual)
    y_pred.append(predicted)

# Calcolo metriche
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

AttributeError: 'Series' object has no attribute 'iteritems'